Decision tree classifier from scratch and python using card algorithm

Initialising training_data which contain different fruits and their features

In [2]:
training_data=[
    ['Green',3,'Mango'],
    ['Yellow',3,'Mango'],
    ['Red',1,'Grape'],
    ['Red',1,'Grape'],
    ['Yellow',3,'Lemon'],
]



 adding Column labels to print the tree

In [3]:
header=["color","diameter","label"]

Define a function to find the unique values in the column in dataset

In [4]:
def unique_vals(rows,col):
    return set([row[col] for row in rows])

Define a function to count the number of each type of example in a dataset

In [30]:
def class_counts(rows):
    counts={}
    for row in rows:
        label=row[-1]
        if label not in counts:
            counts[label]=0
        counts[label]+=1
    return counts

Define a function to test if a value is numeric

In [31]:
def is_numeric(value):
    return isinstance(value,int) or isinstance(value,float)

Now we should define a class named "Question" to partition a dataset. 
           This class just records a 'colum number'(eg:0 for color) and a 'column value'(eg:Green).The match method is used to compare the feature value in an example to the feature value stored in the question

In [29]:
class Question:
    def __init__(self, column, value):
        self.column = column
        self.value = value

    def match(self, example):
        val = example[self.column]
        if isinstance(val, int) or isinstance(val, float):
            return val >= self.value
        else:
            return val == self.value

    def __repr__(self):
        condition = "=="
        if isinstance(self.value, int) or isinstance(self.value, float):
            condition = ">="
        return f"Is {self.column} {condition} {self.value}?"


Define a function to partition the dataset

In [8]:
def partition(rows,question):
    """Partition is a dataset.

    for each row in the dataset,check if it matches the question. If so,add
    it to  'true rows', otherwise,add it to 'false rows'
    """
    true_rows,false_rows=[],[]
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows,false_rows        

Define a function to calculate the Gini Impurity for a list of rows

In [32]:
def gini(rows):
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

Define a function to calculate the information gain by subtracting the weighted impurity of two child nodes from the uncertainity of starting node 

In [33]:
def info_gain(left,right,current_uncertainity):
    p=float(len(left))/(len(left)+len(right))
    return current_uncertainity-p*gini(left)-(1-p)*gini(right)

Define a function to find the best question to ask by iterating over every feature/value and claculating the information gain

In [34]:
def find_best_split(rows):
    best_gain = 0  # keep track of the best info gain
    best_question = None  # keep track of the feature/value that produced it
    current_uncertainity = gini(rows)
    n_features = len(rows[0]) - 1  # number of columns

    for col in range(n_features):
        values = set([row[col] for row in rows])  # unique values in the column

        for val in values:
            question = Question(col, val)

            # Try splitting the dataset
            true_rows, false_rows = partition(rows, question)
            
            # Skip this split if it doesn't divide the dataset
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            # Calculate the info gain from this split
            gain = info_gain(true_rows, false_rows, current_uncertainity)

            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question


Define a class 'Leaf'

In [40]:
class Leaf:
    """A leaf node classifies data.
    This holds a dictionary of class(eg:"Mango")-> number of times
    it appears in the rows from the training data that reach this leaf.
    """
    def __init__(self, rows):
        self.predictions = class_counts(rows)

class Decision_Node:
    """A Decision Node asks a question.
    This holds a reference to the question and to the two child nodes.
    """
    def __init__(self, question, true_branch, false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch
        
def build_tree(rows):
    """Builds the tree."""
    gain, question = find_best_split(rows)
    if gain == 0:
        return Leaf(rows)

    true_rows, false_rows = partition(rows, question)
    true_branch = build_tree(true_rows)
    false_branch = build_tree(false_rows)

    return Decision_Node(question, true_branch, false_branch)#return a question node

Define a function to print the tree

In [41]:
def print_tree(node, spacing=""):
    # Base case: if this is a leaf node, print the predictions
    if isinstance(node, Leaf):
        print(spacing + "Predict", node.predictions)
        return

    # Print the question at this node
    print(spacing + str(node.question))

    # Call this function recursively on the true branch
    print(spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Call this function recursively on the false branch
    print(spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")


Define a classify function to determine whether to fall the true branch or false branch

In [42]:
def classify(row,node):
    if isinstance(node,Leaf):
        return node.predictions

    if node.question.match(row):
        return classify(row,node.true_branch)
    else:
        return classify(row,node.false_branch)

Define a function to print the predictions at a leaf

In [43]:
def print_leaf(counts):
    total=sum(counts.values())*1.0
    probs={}
    for lbl in counts.keys():
        probs[lbl]=str(int(counts[lbl]/total*100))+"%"
    return probs    

In [45]:
if __name__ == '__main__':
    my_tree=build_tree(training_data)
    print_tree(my_tree)

    #Evaluate
    testing_data=[
         ['Green',3,'Mango'],
    ['Yellow',4,'Mango'],
    ['Red',2,'Grape'],
    ['Red',1,'Grape'],
    ['Yellow',3,'Lemon'],
    ]
    for row in testing_data:
        print("Actual:%s. Predicted:%s" %
             (row[-1],print_leaf(classify(row,my_tree))))

Is 1 >= 3?
--> True:
  Is 0 == Yellow?
  --> True:
    Predict {'Mango': 1, 'Lemon': 1}
  --> False:
    Predict {'Mango': 1}
--> False:
  Predict {'Grape': 2}
Actual:Mango. Predicted:{'Mango': '100%'}
Actual:Mango. Predicted:{'Mango': '50%', 'Lemon': '50%'}
Actual:Grape. Predicted:{'Grape': '100%'}
Actual:Grape. Predicted:{'Grape': '100%'}
Actual:Lemon. Predicted:{'Mango': '50%', 'Lemon': '50%'}
